In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import glob
import os
from topmed_manuscript_clean import phenotype_id_to_gene_id
from scipy.stats import fisher_exact as fe
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
import random
import re

random.seed(42)

def shuffle_dict(d):
    """
    Shuffles the key - value relationships of a dictionary
    
    x = {'a': 0, 'b': 1, 'c': 2}
    print(x)
    print(shuffle_dict(x))
    
    """
    keys = list(d.keys())
    values = [d[k] for k in keys]
    random.shuffle(keys)
    return dict(zip(keys, values))


def parse_ensembl_id(x):
    """
    See https://m.ensembl.org/info/genome/stable_ids/index.html:
    ENS[species prefix][feature type prefix][a unique eleven digit number]
    """
    FEATURE_TYPE_PREFICES = ['E', 'FM', 'G', 'GT', 'P', 'R', 'T']
    ENSEMBL_RE = '^ENS(.*?)({})(\d+)\.?(.*?)$'.format('|'.join(FEATURE_TYPE_PREFICES))
    species_prefix, feature_type_prefix, digits, version = re.match(ENSEMBL_RE, x).groups()
    return {'species_prefix': species_prefix, 'feature_type_prefix': feature_type_prefix, 'digits': digits, 'version': version}

    
def strip_version_from_ensembl_gene_id(x, keep_par_y=False):
    """
    ENSG00000280767.3 --> ENSG00000280767
    if keep_par_y: ENSG00000280767.3_PAR_Y --> ENSG00000280767.PAR_Y
    if !keep_par_y: ENSG00000280767.3_PAR_Y --> ENSG00000280767
    """
    x = parse_ensembl_id(x)
    if keep_par_y and 'PAR_Y' in x['version']:
        return 'ENS{species_prefix}{feature_type_prefix}{digits}.PAR_Y'.format(**x)
    else:
        return 'ENS{species_prefix}{feature_type_prefix}{digits}'.format(**x)


def strip_version(x):
    return strip_version_from_ensembl_gene_id(x, keep_par_y=True)


def fishers_exact(variants, control_variants, annotation_table):
    """
    variants: list/array of variant IDs
    control_variants: list/array of control variant IDs
    annotation_table: binary matrix indicating if each variant (row) overlaps each annotation (column). Index must contain all variants and control variants

    returns: fishers exact test results comparing variant to control variant overlap of each annotation
    """
    n_hits_overlapping_annotation = annotation_table.loc[variants].sum()
    n_hits_not_overlapping_annotation = len(variants) - n_hits_overlapping_annotation
    n_controls_overlapping_annotation = annotation_table.loc[control_variants].sum()
    n_controls_not_overlapping_annotation = len(control_variants) - n_controls_overlapping_annotation
    
    fet_tables = {annotation: 
                  np.array([
                      [n_hits_overlapping_annotation[annotation], n_hits_not_overlapping_annotation[annotation]], 
                      [n_controls_overlapping_annotation[annotation], n_controls_not_overlapping_annotation[annotation]]
                      ]) for annotation in annotation_table.columns}
    fet = {annotation: fe(v) for annotation, v in fet_tables.items()}
    
    results = pd.DataFrame([[annotation, v[0], v[1]] for annotation, v in fet.items()], columns=['annotation', 'odds_ratio', 'p'])
    results['n_hits_overlapping_annotation'] = n_hits_overlapping_annotation.loc[results.annotation.to_list()].to_list()
    results['n_hits_not_overlapping_annotation'] = n_hits_not_overlapping_annotation.loc[results.annotation.to_list()].to_list()
    results['n_controls_overlapping_annotation'] = n_controls_overlapping_annotation.loc[results.annotation.to_list()].to_list()
    results['n_controls_not_overlapping_annotation'] = n_controls_not_overlapping_annotation.loc[results.annotation.to_list()].to_list()

    return results

def parse_attribute(attribute_series: pd.Series, attribute_name: str) -> pd.Series:
    """
    Parse the attributes column of a (GENCODE/RefSeq) GTF file.

    Input:
    * a [str]: the attributes element (column 9 of the GTF file)
    * regex [str]: a regular expression that will be iteratively applied to the attribute string to capture attribute key, val pairs. Default should work for GENCODE/RefSeq
    """
    if not isinstance(attribute_series, pd.Series):
        raise TypeError('attribute_series must be a pandas Series')
    if not isinstance(attribute_name, str):
        raise TypeError('attribute_name must be a string')
    
    return attribute_series.str.extract(f'{attribute_name} "(.*?)"')


def gtf_to_df(gtf: str, parse_attributes: list=None) -> pd.DataFrame:
    df = pd.read_csv(gtf, sep='\t', header=None, names=['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes'], comment='#')
    if parse_attributes is not None:
        for a in parse_attributes:
            df[a] = parse_attribute(df.attributes, a)
    return df

PREFIX = 'tables/trans-enrichment-in-cis.'
CLUMPED_TRANS_EQTL = '../work/clump-trans-variants/clump-trans-signals.significant-trans-eqtl-clumped.tsv'
CLUMPED_TRANS_SQTL = '../work/clump-trans-variants/clump-trans-signals.significant-trans-sqtl-clumped.tsv'
CONTROL_SNPS_TRANS_EQTL = glob.glob('../work/control-snps/trans-eqtl/maf005/results/controls/*.controls.txt')
CONTROL_SNPS_TRANS_SQTL = glob.glob('../work/control-snps/trans-sqtl/maf005/results/controls/*.controls.txt')
GTF = '../data/gtf/gencode.v30.GRCh38.ERCC.genes.collapsed_only.gtf.gz'
USE_TISSUES = ['Whole_blood']

/net/snowwhite/home/porchard/anaconda3/envs/2020-09/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/net/snowwhite/home/porchard/anaconda3/envs/2020-09/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
gtf_df = gtf_to_df(GTF, parse_attributes=['gene_id', 'gene_name'])
gtf_df = gtf_df[gtf_df.feature=='gene']
gtf_df.head()

,chrom,source,feature,start,end,score,strand,frame,attributes,gene_id,gene_name
0,chr1,HAVANA,gene,11869,14409,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,DDX11L1
6,chr1,HAVANA,gene,14404,29570,.,-,.,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5,WASH7P
19,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,MIR6859-1
22,chr1,HAVANA,gene,29554,31109,.,+,.,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5,MIR1302-2HG
27,chr1,ENSEMBL,gene,30366,30503,.,+,.,"gene_id ""ENSG00000284332.1""; transcript_id ""EN...",ENSG00000284332.1,MIR1302-2


In [3]:
trans_eqtl = pd.read_csv(CLUMPED_TRANS_EQTL, sep='\t')
trans_eqtl = trans_eqtl.loc[(trans_eqtl.qvalue<=0.05),['clumped_variant_id', 'phenotype_id', 'tissue', 'af']].rename(columns={'clumped_variant_id': 'variant_id'})
trans_eqtl.head()

,variant_id,phenotype_id,tissue,af
0,chr7_50342615_A_G,ENSG00000000938.13,Whole_blood,0.235203
1,chr1_156344836_A_G,ENSG00000002330.13,Whole_blood,0.500775
2,chr6_144036619_C_A,ENSG00000004059.11,Whole_blood,0.055934
3,chr6_122440739_T_C,ENSG00000004478.8,Whole_blood,0.706539
4,chr22_46290431_C_G,ENSG00000004799.8,Whole_blood,0.107530


In [4]:
trans_sqtl = pd.read_csv(CLUMPED_TRANS_SQTL, sep='\t')
trans_sqtl = trans_sqtl.loc[(trans_sqtl.qvalue<=0.05),['clumped_variant_id', 'phenotype_id', 'gene_id', 'tissue', 'af']].rename(columns={'clumped_variant_id': 'variant_id'})
trans_sqtl.head()

,variant_id,phenotype_id,gene_id,tissue,af
0,chr1_156344313_T_C,chr19:35907762:35908175:clu_21928_-:ENSG000000...,ENSG00000011600.11,Whole_blood,0.497753
1,chr7_50360284_G_A,chrX:48467925:48468305:clu_44487_-:ENSG0000001...,ENSG00000017483.15,Whole_blood,0.498528
2,chr3_187117653_G_A,chr4:25256674:25259037:clu_33710_+:ENSG0000003...,ENSG00000038210.13,Whole_blood,0.633561
3,chr6_163408503_T_C,chr11:63903158:63904786:clu_8082_+:ENSG0000007...,ENSG00000072518.20,Whole_blood,0.933065
4,chr6_163408503_T_C,chr11:85983973:85990250:clu_7229_-:ENSG0000007...,ENSG00000073921.18,Whole_blood,0.932987


In [5]:
cis_eqtl = pd.concat([pd.read_csv(f, sep='\t').assign(tissue=os.path.basename(f).split('.')[0]) for f in glob.glob('../data/scan-results/joint/cis-eqtl/susie/maf001/*.cs.txt')])
cis_sqtl = pd.concat([pd.read_csv(f, sep='\t').assign(tissue=os.path.basename(f).split('.')[0]) for f in glob.glob('../data/scan-results/joint/cis-sqtl/susie/maf001/postprocessed/*.by-gene.cs.txt')])
cis_sqtl.head()

,phenotype_id,variant_id,pip,af,cs_id,tissue
0,chr20:50941209:50942031:clu_35052_-:ENSG000000...,chr20_50879604_A_C,0.054397,0.037568,1,Lung
1,chr20:50941209:50942031:clu_35052_-:ENSG000000...,chr20_50882479_T_C,0.055092,0.025174,1,Lung
2,chr20:50941209:50942031:clu_35052_-:ENSG000000...,chr20_50885892_G_C,0.237507,0.019752,1,Lung
3,chr20:50941209:50942031:clu_35052_-:ENSG000000...,chr20_50888836_C_A,0.030341,0.024400,1,Lung
4,chr20:50941209:50942031:clu_35052_-:ENSG000000...,chr20_50892638_C_T,0.079246,0.020139,1,Lung


In [6]:
controls_trans_eqtl = pd.concat([pd.read_csv(f, sep='\t').assign(tissue=os.path.basename(f).replace('.controls.txt', '')) for f in CONTROL_SNPS_TRANS_EQTL])
controls_trans_eqtl = controls_trans_eqtl[controls_trans_eqtl.tissue.isin(USE_TISSUES)]
controls_trans_eqtl = controls_trans_eqtl.groupby(['tissue', 'variant']).head(1)
controls_trans_sqtl = pd.concat([pd.read_csv(f, sep='\t').assign(tissue=os.path.basename(f).replace('.controls.txt', '')) for f in CONTROL_SNPS_TRANS_SQTL])
controls_trans_sqtl = controls_trans_sqtl[controls_trans_sqtl.tissue.isin(USE_TISSUES)]
controls_trans_sqtl = controls_trans_sqtl.groupby(['tissue', 'variant']).head(1)
controls_trans_eqtl.head()

,variant,control_variant,variant_maf,control_variant_maf,tissue
0,chr18_79466098_G_A,chr18_60801235_T_C,0.05562,0.05803,Whole_blood
5,chr18_2646148_A_G,chr18_8927091_A_G,0.07251,0.07282,Whole_blood
10,chr18_76821755_T_C,chr18_74410805_T_G,0.18720,0.19210,Whole_blood
15,chr18_72760883_T_C,chr18_77018215_A_G,0.30860,0.30450,Whole_blood
20,chr18_74608983_G_A,chr18_8792525_C_CT,0.31100,0.31600,Whole_blood


In [7]:
len(controls_trans_eqtl)

614

In [8]:
assert(list(sorted(trans_eqtl[trans_eqtl.tissue=='Whole_blood'].variant_id.unique().tolist())) == list(sorted(controls_trans_eqtl[controls_trans_eqtl.tissue=='Whole_blood'].variant.to_list())))

In [9]:
# make annotation matrices
annotation_matrices = dict()
for tissue in USE_TISSUES:
    tissue_trans_variants_and_controls = controls_trans_eqtl.loc[controls_trans_eqtl.tissue==tissue,['variant', 'control_variant']].values.flatten().tolist()
    tissue_trans_variants_and_controls += controls_trans_sqtl.loc[controls_trans_sqtl.tissue==tissue,['variant', 'control_variant']].values.flatten().tolist()
    tissue_trans_variants_and_controls = list(set(tissue_trans_variants_and_controls))
    m = pd.DataFrame({'variant': tissue_trans_variants_and_controls})
    m['ciseqtl'] = m.variant.isin(cis_eqtl[cis_eqtl.tissue==tissue].variant_id.unique())
    m['cissqtl'] = m.variant.isin(cis_sqtl[cis_sqtl.tissue==tissue].variant_id.unique())
    m['ciseqtl_or_cissqtl'] = m.ciseqtl | m.cissqtl
    annotation_matrices[tissue] = m.set_index('variant').astype(int)

In [10]:
annotation_matrices['Whole_blood'].head()

,ciseqtl,cissqtl,ciseqtl_or_cissqtl
variant,,,
chr2_49050684_G_A,0,0,0
chr16_10815232_C_A,0,0,0
chrX_43226118_G_T,0,0,0
chr3_77722227_A_G,0,0,0
chr19_12419069_C_G,0,0,0


In [11]:
assert(all((annotation_matrices['Whole_blood'].loc[:,['ciseqtl', 'cissqtl']].sum(axis=1)>0).astype(int) == annotation_matrices['Whole_blood'].loc[:,'ciseqtl_or_cissqtl']))

In [12]:
# test for enrichment w/ cis-e/sQTL
results = []
for tissue in USE_TISSUES:
    trans_eqtl_enrichment = fishers_exact(controls_trans_eqtl[controls_trans_eqtl.tissue==tissue].variant.to_list(), controls_trans_eqtl[controls_trans_eqtl.tissue==tissue].control_variant.to_list(), annotation_matrices[tissue])
    trans_sqtl_enrichment = fishers_exact(controls_trans_sqtl[controls_trans_sqtl.tissue==tissue].variant.to_list(), controls_trans_sqtl[controls_trans_sqtl.tissue==tissue].control_variant.to_list(), annotation_matrices[tissue])
    results.append(trans_eqtl_enrichment.assign(tissue=tissue, modality='transeqtl'))
    results.append(trans_sqtl_enrichment.assign(tissue=tissue, modality='transsqtl'))
results = pd.concat(results)
results.to_csv(f'{PREFIX}basic-trans-enrichment-in-cis.txt', sep='\t', index=False)

In [13]:
results

,annotation,odds_ratio,p,n_hits_overlapping_annotation,n_hits_not_overlapping_annotation,n_controls_overlapping_annotation,n_controls_not_overlapping_annotation,tissue,modality
0,ciseqtl,7.035625,4.978788e-27,163,451,30,584,Whole_blood,transeqtl
1,cissqtl,3.109282,1.398202e-07,82,532,29,585,Whole_blood,transeqtl
2,ciseqtl_or_cissqtl,5.781351,6.342585e-29,202,412,48,566,Whole_blood,transeqtl
0,ciseqtl,19.565217,2.188607e-04,15,46,1,60,Whole_blood,transsqtl
1,cissqtl,5.105769,5.366637e-02,9,52,2,59,Whole_blood,transsqtl
2,ciseqtl_or_cissqtl,8.746032,2.529502e-04,19,42,3,58,Whole_blood,transsqtl


In [14]:
results[(results.tissue=='Whole_blood') & (results.modality=='transeqtl') & (results.annotation=='ciseqtl_or_cissqtl')]

,annotation,odds_ratio,p,n_hits_overlapping_annotation,n_hits_not_overlapping_annotation,n_controls_overlapping_annotation,n_controls_not_overlapping_annotation,tissue,modality
2,ciseqtl_or_cissqtl,5.781351,6.342585e-29,202,412,48,566,Whole_blood,transeqtl


In [15]:
x = results[(results.tissue=='Whole_blood') & (results.modality=='transeqtl') & (results.annotation=='ciseqtl_or_cissqtl')]
y = results[(results.tissue=='Whole_blood') & (results.modality=='transsqtl') & (results.annotation=='ciseqtl_or_cissqtl')]
assert(len(x) == 1)
assert(len(y) == 1)
print('{}% of unique trans-eVariants overlap at least one cis-eQTL or cis-sQTL credible set in one tissue ({}% for trans-sVariants)'.format(
    round(100*x.n_hits_overlapping_annotation.values[0] / (x.n_hits_overlapping_annotation.values[0] + x.n_hits_not_overlapping_annotation.values[0]), 1),
    round(100*y.n_hits_overlapping_annotation.values[0] / (y.n_hits_overlapping_annotation.values[0] + y.n_hits_not_overlapping_annotation.values[0]), 1)
))

32.9% of unique trans-eVariants overlap at least one cis-eQTL or cis-sQTL credible set in one tissue (31.1% for trans-sVariants)


In [16]:
# compare enrichment in the case where trans-eVariant is trans-eVariant for more than 1 trans-eGene vs only 1 trans-eGene
# results = []

# for tissue in USE_TISSUES:
#     trans_evariants = trans_eqtl[trans_eqtl.tissue==tissue].variant_id.value_counts()
#     trans_eqtl_for_more_than_one_gene = trans_evariants[trans_evariants>1].index.tolist()
#     trans_eqtl_for_one_gene = trans_evariants[trans_evariants==1].index.tolist()

#     trans_svariants = trans_sqtl[trans_sqtl.tissue==tissue].variant_id.value_counts()
#     trans_sqtl_for_more_than_one_gene = trans_svariants[trans_svariants>1].index.tolist()
#     trans_sqtl_for_one_gene = trans_svariants[trans_svariants==1].index.tolist()

#     trans_eqtl_enrichment = fishers_exact(trans_eqtl_for_more_than_one_gene, trans_eqtl_for_one_gene, annotation_matrices[tissue])
#     trans_sqtl_enrichment = fishers_exact(trans_sqtl_for_more_than_one_gene, trans_sqtl_for_one_gene, annotation_matrices[tissue])
#     results.append(trans_eqtl_enrichment.assign(tissue=tissue, modality='transeqtl'))
#     results.append(trans_sqtl_enrichment.assign(tissue=tissue, modality='transsqtl'))

# results = pd.concat(results)
# results.to_csv(f'{PREFIX}trans-enrichment-in-cis-more-than-one-transegene-vs-only-one-transgene.txt', sep='\t', index=False)

In [17]:
# tmp = trans_eqtl.groupby(['tissue', 'variant_id', 'af']).phenotype_id.nunique().rename('n_egenes').reset_index()
# tmp['maf'] = np.minimum(tmp.af, 1-tmp.af)
# tmp['evariant_for_more_than_one_gene'] = tmp.n_egenes > 1
# tmp.head()

In [18]:
# fig, ax = plt.subplots()

# sns.stripplot(x='evariant_for_more_than_one_gene', y='maf', data=tmp, ax=ax)
# sns.boxplot(x='evariant_for_more_than_one_gene', y='maf', data=tmp, ax=ax, color='white')

In [19]:
# TODO: subset to genes that were considered in the paper that selected TFs. If gene isn't in the list, not clear if it's because it was considered and wasn't a TF or because it wasn't considered at all
# TODO: subset to cis tested genes only?
tf_list = set(pd.read_csv('/net/topmed10/working/porchard/rnaseq/data/tfs/TFs_Ensembl_v_1.01.txt', sep='\t', header=None)[0])

gene_is_tf = {i: strip_version(i) in tf_list for i in gtf_df.gene_id.unique() if 'ENS' in i}
# not losing much by subsetting to genes that appear in the GO mappings
# tmp = cis_eqtl.phenotype_id.isin(set(gene_is_tf.keys())).value_counts()
# tmp/tmp.sum()

In [20]:
cis_eqtl['gene_id'] = cis_eqtl.phenotype_id
cis_sqtl['gene_id'] = cis_sqtl.phenotype_id.map(phenotype_id_to_gene_id)

In [21]:
# use only unique variants -- otherwise, if trans-eVariant w/ many trans-eGenes becomes cis-eQTL for TF in subset of permutations, get weird upper tail density
# Amongst genes w/ at least one eQTL credible set, TF genes don't have more credible sets or bigger credible sets
# so can use permutation test amongst cis-eGenes to test for significance here
# use whole blood only?

REPS = 1000

observed_all = []
permuted_all = []

for tissue in USE_TISSUES:
    for trans in ['trans_eqtl', 'trans_sqtl']:
        for cis in ['cis_eqtl', 'cis_sqtl']:
            trans_df = trans_eqtl if trans == 'trans_eqtl' else trans_sqtl
            cis_df = cis_eqtl if cis == 'cis_eqtl' else cis_sqtl
            tissue_trans_variants = trans_df[(trans_df.tissue==tissue)].variant_id.unique()
            tissue_trans_variants_to_cis_genes = cis_df.loc[(cis_df.tissue==tissue) & (cis_df.variant_id.isin(tissue_trans_variants)),['variant_id', 'gene_id']].drop_duplicates()

            cis_egene_is_tf = {i: gene_is_tf[i] for i in cis_df[cis_df.tissue==tissue].gene_id.unique() if i in gene_is_tf}
            tissue_trans_variants_to_cis_genes['gene_is_tf'] = tissue_trans_variants_to_cis_genes.gene_id.map(cis_egene_is_tf).astype(bool)

            observed = tissue_trans_variants_to_cis_genes.groupby('variant_id').gene_is_tf.max().sum()
            observed_all.append([tissue, trans, cis, observed])
            
            for i in range(REPS):
                if i % 100 == 0:
                    logging.info(f'Permutation {i} of {REPS}')
                # shuffle gene --> is_tf assignments
                tmp = shuffle_dict(cis_egene_is_tf)
                tissue_trans_variants_to_cis_genes['gene_is_tf'] = tissue_trans_variants_to_cis_genes.gene_id.map(tmp).astype(bool)
                stat = tissue_trans_variants_to_cis_genes.groupby('variant_id').gene_is_tf.max().sum()
                permuted_all.append([tissue, trans, cis, stat])

observed = pd.DataFrame(observed_all, columns=['tissue', 'trans_modality', 'cis_modality', 'n']).assign(c='observed')
permuted = pd.DataFrame(permuted_all, columns=['tissue', 'trans_modality', 'cis_modality', 'n']).assign(c='permuted')
cisgene_is_tf_permutations = pd.concat([observed, permuted])
cisgene_is_tf_permutations.to_csv(f'{PREFIX}cisgene_is_tf_permutations.tsv', sep='\t', index=False)

2024-12-03 14:16:07,477 - INFO: Permutation 0 of 1000
2024-12-03 14:16:08,695 - INFO: Permutation 100 of 1000
2024-12-03 14:16:09,914 - INFO: Permutation 200 of 1000
2024-12-03 14:16:11,122 - INFO: Permutation 300 of 1000
2024-12-03 14:16:12,332 - INFO: Permutation 400 of 1000
2024-12-03 14:16:13,546 - INFO: Permutation 500 of 1000
2024-12-03 14:16:14,742 - INFO: Permutation 600 of 1000
2024-12-03 14:16:15,955 - INFO: Permutation 700 of 1000
2024-12-03 14:16:17,167 - INFO: Permutation 800 of 1000
2024-12-03 14:16:18,387 - INFO: Permutation 900 of 1000
2024-12-03 14:16:20,096 - INFO: Permutation 0 of 1000
2024-12-03 14:16:20,736 - INFO: Permutation 100 of 1000
2024-12-03 14:16:21,373 - INFO: Permutation 200 of 1000
2024-12-03 14:16:22,016 - INFO: Permutation 300 of 1000
2024-12-03 14:16:22,657 - INFO: Permutation 400 of 1000
2024-12-03 14:16:23,292 - INFO: Permutation 500 of 1000
2024-12-03 14:16:23,929 - INFO: Permutation 600 of 1000
2024-12-03 14:16:24,570 - INFO: Permutation 700 of 1